# Capstone Project

## Week 5 Final Code

### Recommending to a Property Developer where to built a Shopping Mall in Mumbai

* Build a dataframe of neighborhoods in Mumbai, India by web scraping the data from Wikipedia page
* Get the geographical coordinates of the neighborhoods
* Obtain the venue data for the neighborhoods from Foursquare API
* Explore and cluster the neighborhoods
* Select the best cluster to open a new shopping mall

### 1. Import Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library
import re

print("Libraries imported.")

Libraries imported.


### 2. Scrap data from Wikipedia page into a DataFrame

In [2]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Mumbai").text

In [3]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [4]:
regex = re.compile('^tocsection-')
content_lis = soup.find_all('li', attrs={'class': regex})
print(content_lis)

[<li class="toclevel-1 tocsection-1"><a href="#Western_Suburbs"><span class="tocnumber">1</span> <span class="toctext">Western Suburbs</span></a>
<ul>
<li class="toclevel-2 tocsection-2"><a href="#Andheri"><span class="tocnumber">1.1</span> <span class="toctext">Andheri</span></a></li>
<li class="toclevel-2 tocsection-3"><a href="#Bandra"><span class="tocnumber">1.2</span> <span class="toctext">Bandra</span></a></li>
<li class="toclevel-2 tocsection-4"><a href="#Borivali"><span class="tocnumber">1.3</span> <span class="toctext">Borivali</span></a></li>
<li class="toclevel-2 tocsection-5"><a href="#Dahisar"><span class="tocnumber">1.4</span> <span class="toctext">Dahisar</span></a></li>
<li class="toclevel-2 tocsection-6"><a href="#Goregaon"><span class="tocnumber">1.5</span> <span class="toctext">Goregaon</span></a></li>
<li class="toclevel-2 tocsection-7"><a href="#Jogeshwari"><span class="tocnumber">1.6</span> <span class="toctext">Jogeshwari</span></a></li>
<li class="toclevel-2 toc

In [5]:
# append the data into the list
neighborhoodList = []
for li in content_lis:
    neighborhoodList.append(li.getText().split('\n')[0])
print(neighborhoodList)

['1 Western Suburbs', '1.1 Andheri', '1.2 Bandra', '1.3 Borivali', '1.4 Dahisar', '1.5 Goregaon', '1.6 Jogeshwari', '1.7 Juhu', '1.8 Kandivali west', '1.9 Khar', '1.10 Malad', '1.11 Santacruz', '1.12 Vasai', '1.13 Virar', '1.14 Vile Parle', '2 Eastern Suburbs', '2.1 Bhandup', '2.2 Ghatkopar', '2.3 Kanjurmarg', '2.4 Kurla', '2.5 Mulund', '2.6 Powai', '2.7 Vidyavihar', '2.8 Vikhroli', '3 Harbour Suburbs', '3.1 Chembur', '3.2 Govandi', '3.3 Mankhurd', '3.4 Trombay', '4 South Mumbai', '4.1 Antop Hill', '4.2 Byculla', '4.3 Colaba', '4.4 Dadar', '4.5 Fort', '4.6 Girgaon', '4.7 Kalbadevi', '4.8 Kamathipura', '4.9 Matunga', '4.10 Parel', '4.11 Tardeo', '5 Other', '6 References']


In [6]:
# create a new DataFrame from the list
mum_df = pd.DataFrame({"Names": neighborhoodList})

mum_df.head()

,Names
0,1 Western Suburbs
1,1.1 Andheri
2,1.2 Bandra
3,1.3 Borivali
4,1.4 Dahisar


In [7]:

# new data frame with split value columns 
new = mum_df["Names"].str.split(" ", n = 1, expand = True) 
  
# making separate first name column from new data frame 
mum_df["index"]= new[0] 
  
# making separate last name column from new data frame 
mum_df["Neighborhood"]= new[1] 

mum_df.drop(columns =["index"], inplace = True) 
mum_df.drop(columns =["Names"], inplace = True)
mum_df.head()

,Neighborhood
0,Western Suburbs
1,Andheri
2,Bandra
3,Borivali
4,Dahisar


In [8]:
# print the number of rows of the dataframe
mum_df.shape

(43, 1)

### 3. Get the geographical coordinates

In [9]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Mumbai, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [10]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in mum_df["Neighborhood"].tolist() ]

In [11]:
coords

[[19.19700000000006, 72.82763000000006],
 [19.11848309908247, 72.84177419095158],
 [19.054220000000043, 72.84019000000006],
 [19.229360000000042, 72.85751000000005],
 [19.250030000000038, 72.85908000000006],
 [19.164550000000077, 72.84946000000008],
 [19.13790000000006, 72.84941000000003],
 [19.01493000000005, 72.84522000000004],
 [19.207110000000057, 72.83492000000007],
 [19.069120000000055, 72.84643000000005],
 [19.186550000000068, 72.84836000000007],
 [19.081770000000063, 72.84205000000003],
 [19.07934000000006, 72.83916000000005],
 [19.08068000000003, 72.84353000000004],
 [19.100580000000036, 72.84377000000006],
 [19.005611262064793, 72.85584595073745],
 [19.13498542205837, 72.94007212261138],
 [19.086476606699875, 72.90895177728078],
 [19.131400000000042, 72.93565000000007],
 [18.95373692701983, 72.84031150906404],
 [19.171850000000063, 72.95564000000007],
 [19.123110000000054, 72.90944000000007],
 [19.023261171244744, 72.84389992492353],
 [19.115990000000068, 72.92926000000006],


In [12]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [13]:
# merge the coordinates into the original dataframe
mum_df['Latitude'] = df_coords['Latitude']
mum_df['Longitude'] = df_coords['Longitude']

In [14]:
mum_df.head()

,Neighborhood,Latitude,Longitude
0,Western Suburbs,19.197000,72.827630
1,Andheri,19.118483,72.841774
2,Bandra,19.054220,72.840190
3,Borivali,19.229360,72.857510
4,Dahisar,19.250030,72.859080


In [15]:
# the following steps are done to remove unwanted data from the dataframe
mum_df.drop([0, 15, 24, 29, 41,42], inplace=True)

In [16]:
mum_df.shape

(37, 3)

In [17]:
mum_df = mum_df.reset_index()

In [18]:
del mum_df['index']
mum_df.head()

,Neighborhood,Latitude,Longitude
0,Andheri,19.118483,72.841774
1,Bandra,19.054220,72.840190
2,Borivali,19.229360,72.857510
3,Dahisar,19.250030,72.859080
4,Goregaon,19.164550,72.849460


In [19]:
# save the DataFrame as CSV file
mum_df.to_csv("mum_df.csv", index=False)

### 4. Create a map of Mumbai with neighborhoods superimposed on top

In [20]:
# get the coordinates of Mumbai
address = 'Mumbai, India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mumbai, India 18.9387711, 72.8353355.


In [21]:
# create map of Mumbai using latitude and longitude values
map_mum = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(mum_df['Latitude'], mum_df['Longitude'], mum_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_mum)  
    
map_mum

In [22]:
# save the map as HTML file
map_mum.save('map_mum.html')

### 5. Use the Foursquare API to explore the neighborhoods

In [23]:
# define Foursquare Credentials and Version
CLIENT_ID = 'HQEQE1TZITPAIU2XYGJMHUS53GYN3KQXQJFHNR0TKGG45URY' 
CLIENT_SECRET = 'H4B5L05HCWR0NDBLPQBATQZPFF1N3R01MJKBKMTZMLWSTUVJ' 
VERSION = '20190527' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HQEQE1TZITPAIU2XYGJMHUS53GYN3KQXQJFHNR0TKGG45URY
CLIENT_SECRET:H4B5L05HCWR0NDBLPQBATQZPFF1N3R01MJKBKMTZMLWSTUVJ


#### Now, let's get the top 100 venues that are within a radius of 2000 meters.

In [24]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(mum_df['Latitude'], mum_df['Longitude'], mum_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [25]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(2995, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Andheri,19.118483,72.841774,Merwans Cake shop,19.119300,72.845418,Bakery
1,Andheri,19.118483,72.841774,Naturals,19.111204,72.837255,Ice Cream Shop
2,Andheri,19.118483,72.841774,Radha Krishna Veg Restaurant,19.115130,72.843060,Indian Restaurant
3,Andheri,19.118483,72.841774,Shawarma Factory,19.124591,72.840398,Falafel Restaurant
4,Andheri,19.118483,72.841774,Starbucks Coffee : A Tata Alliance,19.114569,72.836205,Coffee Shop


#### Let's check how many venues were returned for each neighorhood

In [26]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Andheri,100,100,100,100,100,100
Antop Hill,78,78,78,78,78,78
Bandra,100,100,100,100,100,100
Bhandup,22,22,22,22,22,22
Borivali,96,96,96,96,96,96
Byculla,44,44,44,44,44,44
Chembur,100,100,100,100,100,100
Colaba,100,100,100,100,100,100
Dadar,100,100,100,100,100,100


#### Let's find out how many unique categories can be curated from all the returned venues

In [27]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 200 uniques categories.


In [28]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Bakery', 'Ice Cream Shop', 'Indian Restaurant',
       'Falafel Restaurant', 'Coffee Shop', 'Pub', 'Pizza Place',
       'Juice Bar', 'Fast Food Restaurant', 'Snack Place',
       'Sandwich Place', 'Café', 'Multiplex', 'Breakfast Spot',
       'Cocktail Bar', 'Electronics Store', 'Seafood Restaurant',
       'Chinese Restaurant', 'American Restaurant', 'Bar',
       'Department Store', 'Diner', 'BBQ Joint', 'Lounge',
       'Asian Restaurant', 'Residential Building (Apartment / Condo)',
       'Spa', 'Vegetarian / Vegan Restaurant', 'Liquor Store',
       "Women's Store", 'Gym / Fitness Center', 'Food Truck',
       'Smoke Shop', 'Athletics & Sports', 'Fish Market',
       'Martial Arts Dojo', 'Tea Room', 'Hotel', 'Dessert Shop',
       'Gourmet Shop', 'Park', 'Korean Restaurant', 'Restaurant',
       'Sports Club', 'Deli / Bodega', 'Indie Movie Theater',
       'German Restaurant', 'Salad Place', 'College Auditorium',
       'Modern European Restaurant'], dtype=object)

In [29]:
# check if the results contain "Shopping Mall"
"Shopping Mall" in venues_df['VenueCategory'].unique()

True

### 6. Analyze Each Neighborhood

In [30]:
# one hot encoding
mum_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
mum_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [mum_onehot.columns[-1]] + list(mum_onehot.columns[:-1])
mum_onehot = mum_onehot[fixed_columns]

print(mum_onehot.shape)
mum_onehot.head()

(2995, 201)


,Neighborhoods,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Antique Shop,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Bengali Restaurant,Big Box Store,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Burger Joint,Burrito Place,Bus Station,Cafeteria,Café,Chaat Place,Cheese Shop,Chinese Restaurant,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Auditorium,College Gym,Comedy Club,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Eastern European Restaurant,Electronics Store,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,General Entertainment,German Restaurant,Gift Shop,Goan Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Historic Site,History Museum,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Mountain,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,National Park,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Other Great Outdoors,Outdoors & Recreation,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Racetrack,Recreation Center,Residential Building (Apartment / Condo),Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Toll Booth,Toy / Game Store,Track,Trail,Train Station,Vegetarian / Vegan Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Zoo
0,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [31]:
mum_grouped = mum_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(mum_grouped.shape)
mum_grouped

(37, 201)


,Neighborhoods,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Antique Shop,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Bengali Restaurant,Big Box Store,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Burger Joint,Burrito Place,Bus Station,Cafeteria,Café,Chaat Place,Cheese Shop,Chinese Restaurant,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Auditorium,College Gym,Comedy Club,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Eastern European Restaurant,Electronics Store,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,General Entertainment,German Restaurant,Gift Shop,Goan Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Historic Site,History Museum,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Mountain,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,National Park,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Other Great Outdoors,Outdoors & Recreation,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Racetrack,Recreation Center,Residential Building (Apartment / Condo),Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Toll Booth,Toy / Game Store,Track,Trail,Train Station,Vegetarian / Vegan Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Zoo
0,Andheri,0.000000,0.00,0.000000,0.010000,0.000000,0.000000,0.000000,0.020000,0.010000,0.000000,0.010000,0.00,0.020000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.010000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.050000,0.00,0.000000,0.050000,0.000000,0.00,0.03,0.030000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.010000,0.00,0.000000,0.000000,0.000000,0.020000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.00,0.00,0.010000,0.00,0.00,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.040000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.040000,0.160000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.00,0.000000,0.010000,0.020000,0.000000,0.000000,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00

In [32]:
len(mum_grouped[mum_grouped["Shopping Mall"] > 0])

14

#### Create a new DataFrame for Shopping Mall data only

In [33]:
mum_mall = mum_grouped[["Neighborhoods","Shopping Mall"]]

In [34]:
mum_mall.head()

,Neighborhoods,Shopping Mall
0,Andheri,0.000000
1,Antop Hill,0.012821
2,Bandra,0.000000
3,Bhandup,0.045455
4,Borivali,0.020833


### 7. Cluster Neighborhoods

Run k-means to cluster the neighborhoods in Mumbai into 3 clusters.

In [45]:
# set number of clusters
kclusters = 3

mum_clustering = mum_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mum_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 1, 0, 2, 1, 0, 1, 0, 0, 0])

In [46]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
mum_merged = mum_mall.copy()

# add clustering labels
mum_merged["Cluster Labels"] = kmeans.labels_

In [47]:
mum_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
mum_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Andheri,0.000000,0
1,Antop Hill,0.012821,1
2,Bandra,0.000000,0
3,Bhandup,0.045455,2
4,Borivali,0.020833,1


In [48]:
# merge mumbai_grouped with mumbai_data to add latitude/longitude for each neighborhood
mum_merged = mum_merged.join(mum_df.set_index("Neighborhood"), on="Neighborhood")

print(mum_merged.shape)
mum_merged.head()

(37, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Andheri,0.000000,0,19.118483,72.841774
1,Antop Hill,0.012821,1,19.026350,72.866340
2,Bandra,0.000000,0,19.054220,72.840190
3,Bhandup,0.045455,2,19.134985,72.940072
4,Borivali,0.020833,1,19.229360,72.857510


In [49]:
# sort the results by Cluster Labels
print(mum_merged.shape)
mum_merged.sort_values(["Cluster Labels"], inplace=True)
mum_merged

(37, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Andheri,0.000000,0,19.118483,72.841774
32,Vasai,0.000000,0,19.079340,72.839160
31,Trombay,0.000000,0,19.019000,72.897990
30,Tardeo,0.000000,0,18.972430,72.814830
29,Santacruz,0.000000,0,19.081770,72.842050
24,Mankhurd,0.000000,0,19.048530,72.932200
22,Kurla,0.000000,0,18.953737,72.840312
21,Khar,0.000000,0,19.069120,72.846430
19,Kandivali west,0.000000,0,19.207110,72.834920
35,Vile Parle,0.000000,0,19.100580,72.843770


#### Finally, let's visualize the resulting clusters

In [50]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mum_merged['Latitude'], mum_merged['Longitude'], mum_merged['Neighborhood'], mum_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [51]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

### 8. Examine Clusters

#### Cluster 0

In [52]:
mum_merged.loc[mum_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Andheri,0.0,0,19.118483,72.841774
32,Vasai,0.0,0,19.079340,72.839160
31,Trombay,0.0,0,19.019000,72.897990
30,Tardeo,0.0,0,18.972430,72.814830
29,Santacruz,0.0,0,19.081770,72.842050
24,Mankhurd,0.0,0,19.048530,72.932200
22,Kurla,0.0,0,18.953737,72.840312
21,Khar,0.0,0,19.069120,72.846430
19,Kandivali west,0.0,0,19.207110,72.834920
35,Vile Parle,0.0,0,19.100580,72.843770


#### Cluster 1

In [53]:
mum_merged.loc[mum_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
28,Powai,0.011111,1,19.123110,72.909440
33,Vidyavihar,0.010000,1,19.023261,72.843900
1,Antop Hill,0.012821,1,19.026350,72.866340
11,Ghatkopar,0.024691,1,19.086477,72.908952
4,Borivali,0.020833,1,19.229360,72.857510
25,Matunga,0.010101,1,19.027340,72.850480
13,Goregaon,0.020000,1,19.164550,72.849460
6,Chembur,0.010000,1,19.062200,72.902420


#### Cluster 2

In [54]:
mum_merged.loc[mum_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
26,Mulund,0.040541,2,19.171850,72.955640
3,Bhandup,0.045455,2,19.134985,72.940072
23,Malad,0.042105,2,19.186550,72.848360
34,Vikhroli,0.033898,2,19.115990,72.929260
27,Parel,0.030000,2,18.995660,72.839070
20,Kanjurmarg,0.033333,2,19.131400,72.935650


## Observation

Most of the shopping malls are concentrated in the Northern arears of Mumbai city, with the highest number in cluster 1 and moderate number in cluster 2. On the other hand, cluster 0 has very low number of shopping mall in the neighborhoods. This represents a great opportunity and high potential areas to open new shopping malls as there is very little to no competition from existing malls. Meanwhile, shopping malls in cluster 1 are likely suffering from intense competition due to oversupply and high concentration of shopping malls. From another perspective, this also shows that the oversupply of shopping malls mostly happened in the developed parts like Thane in Mumbai city, with the suburb areas like South Mumbai still have very few shopping malls. Therefore, this project recommends property developers to capitalize on these findings to open new shopping malls in neighborhoods in cluster 0 with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new shopping malls in neighborhoods in cluster 2 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 1 which already have high concentration of shopping malls and suffering from intense competition.